<a href="https://www.kaggle.com/code/ludmiladias/multiclasses-unet?scriptVersionId=142900763" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<div id="title"><h1 style="color:white;background:#62909d;border-radius:5px;padding:30px;font-family:'Sans-Serif', cursive;font-size:50px;text-align:center">U-Net Multiclasses Training</h1></div>

Projeto desenvolvido usando a rede de segmentação U-NET, biblioteca Keras e Tensorflow.
O Road Mapper DNN tem como objetivo gerar mapas de estrada com segmentação das faixas de sinalização utilizada pelos veículos, tendo como entrada mapas de remissão gerados pelo LIDAR - sensor laser que faz parte do sistema de carros autônomos como o ASTRO da Lume Robotics.


Input | Output
------|--------
![](https://github.com/LCAD-UFES/carmen_lcad/blob/master/src/road_mapper/data/i7705600_-338380.png?raw=true)|![](https://github.com/LCAD-UFES/carmen_lcad/blob/master/src/road_mapper/data/r7705600_-338380_map_1_6.png?raw=true)



## <div id="summary">**<font color="#62909d" size="5">Tabela de Conteúdos</font>**</div>

**<font size="2"><a href="#chap1">1. Instalar Pacotes</a></font>**
**<br><font size="2"><a href="#chap2">2. Configurações Iniciais</a></font>**
**<br><font size="2"><a href="#chap3">3. Tratamento de Dados</a></font>**
**<br><font size="2"><a href="#chap4">4. Compilação e Treinamento da Rede</a></font>**
**<br><font size="2"><a href="#chap5">5. Analisando os Resultados</a></font>**
**<br><font size="2"><a href="#chap6">6. Realizando Testes</a></font>**

<div id="chap1"><h1 style="color:white;background:#62909d;border-radius:5px;padding:30px;font-family:'Sans-Serif', cursive;font-size:50px;text-align:center">Instalar Pacotes</h1></div>


In [ ]:
%pip install opencv-python
%pip install tensorflow
%pip install numpy
%pip install tqdm
%pip install scikit-image
%pip install scikit-learn
%pip install matplotlib


<div id="chap2"><h1 style="color:white;background:#62909d;border-radius:5px;padding:30px;font-family:'Sans-Serif', cursive;font-size:30px;text-align:center">Configurações Iniciais</h1></div>


In [ ]:
import os
import glob

import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
from skimage.io import imread, imshow
from skimage.transform import resize
import tensorflow as tf

from keras import backend as K



In [ ]:
SIZE_X = 128 
SIZE_Y = 128

TRAIN_PATH = 'DATASET/treino/'
TEST_PATH_HIGHWAY = 'DATASET/teste/highway/'
TEST_PATH_UFES = 'DATASET/teste/ufes/'

IMG_CHANNELS = 1
NUM_CLASSES = 17  # Número de classes

<div id="chap3"><h1 style="color:white;background:#62909d;border-radius:5px;padding:30px;font-family:'Sans-Serif', cursive;font-size:50px;text-align:center">Tratamento de Dados</h1></div>

Pule essa etapa se você já tiver seus dados prontos em um CSV.

**<font color="#62909d" size="5">Importando os arquivos</font>**


In [ ]:
# Obter uma lista das subpastas
train_ids = next(os.walk(TRAIN_PATH))[1]
test_ids_hw = next(os.walk(TEST_PATH_HIGHWAY))[1]
test_ids_uf = next(os.walk(TEST_PATH_UFES))[1]

In [ ]:
X_train = np.zeros((len(train_ids), SIZE_Y, SIZE_X, IMG_CHANNELS), dtype = np.uint8)
Y_train = np.zeros((len(train_ids), SIZE_Y, SIZE_X), dtype = np.uint8)


X_test = np.zeros((len(test_ids_hw), SIZE_Y, SIZE_X, IMG_CHANNELS), dtype=np.uint8)
Y_test = np.zeros((len(test_ids_hw), SIZE_Y, SIZE_X), dtype = np.uint8)


In [ ]:
# CARREGA IMAGENS PARA ARRAY DE TREINO
def imgs_to_array(TRAIN_PATH, type_data):
    if type_data == 'I':
        
        for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)):  
            
            path = TRAIN_PATH + id_
            
            # Read the image and keep the first IMG_CHANNELS channels
            img = imread(path + '/images/i' + id_ + '.png', 0) 
            
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            
            img = np.expand_dims(img, axis=2)
            
            # Resize the image to SIZE_Y x SIZE_X while keeping the pixel values (mode='constant')
            img = resize(img, (SIZE_Y, SIZE_X), mode='constant', preserve_range=True)  
            
            # Store the processed image into the X_train array at index n
            X_train[n]=img
            
        return X_train

    elif type_data == 'M':
    # Loop through all train_ids
        for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)):  
            
            path = TRAIN_PATH + id_
            
            # Read the mask file
            mask = imread(path + '/masks/r' + id_ + '.png', 0)
            
            mask = cv2.resize(mask, (SIZE_Y, SIZE_X), interpolation = cv2.INTER_NEAREST)
            
                
            # Store the processed mask into the Y_train array at index n
            Y_train[n] = mask
            
        return Y_train
    
    return []

In [ ]:
image_dataset = imgs_to_array(TRAIN_PATH, type_data='I') # 'I' - Imagens de input
mask_dataset = imgs_to_array(TRAIN_PATH, type_data='M') # 'M' - Masks do Ground Truth

In [ ]:
# SALVAR ARRAYS DAS IMAGENS

import pickle
try:  
    arquivo = open("bin/image_dataset.bin", "wb")
    pickle.dump(image_dataset, arquivo)
    arquivo.close()
except:
    print("Problemas com o arquivo treino.")

try:  
    arquivo = open("bin/mask_dataset.bin", "wb")
    pickle.dump(mask_dataset, arquivo)
    arquivo.close()
except:
    print("Problemas com o arquivo teste.")

In [ ]:
#  LOAD DOS ARRAYS SALVOS

import pickle

try:
    arquivo = open("/kaggle/input/road-mapper-dataset/image_dataset.bin", "rb")
    image_dataset = pickle.load(arquivo)
    arquivo.close()
except:
    print("Problemas com o arquivo treinox.")
    
try:
    arquivo = open("/kaggle/input/road-mapper-dataset/mask_dataset.bin", "rb")
    mask_dataset = pickle.load(arquivo)
    arquivo.close()
except:
    print("Problemas com o arquivo treinoy.")

**<font color="#62909d" size="5">Normalização e tratamento dos arrays</font>**


In [ ]:
print("Image data shape is: ", image_dataset.shape)
print("Mask data shape is: ", mask_dataset.shape)
print("Max pixel value in image is: ", image_dataset.max())
print("Labels in the mask are : ", np.unique(mask_dataset))

In [ ]:
# Normalizar dados do X
image_dataset = normalize(image_dataset, axis=1)

In [ ]:
# Realizar o Label Encoder
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
n, h, w = image_dataset.shape
train_masks_reshaped = image_dataset.reshape(-1,1)
train_masks_reshaped_encoded = labelencoder.fit_transform(train_masks_reshaped)
train_masks_encoded_original_shape = train_masks_reshaped_encoded.reshape(n, h, w)
train_masks_input = np.expand_dims(train_masks_encoded_original_shape, axis=3)
print(train_masks_input.shape)

In [ ]:
#Dividir dados de treino e teste
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(image_dataset, train_masks_input, test_size = 0.2, random_state = 42)

In [ ]:
print("Os valores das classes nesse dataset são ... ", np.unique(y_train))

In [ ]:
# Gerar pesos das classes (nova implementação em processo)
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight(class_weight = 'balanced',
                                                 classes = np.unique(train_masks_reshaped_encoded),
                                                 y = train_masks_reshaped_encoded)
print("Class weights are...:", class_weights)

**<font color="#62909d" size="5">Salvar arquivo como CSV</font>**


In [ ]:
import csv
import pandas as pd

def salvar_csv(X, Y, file_name):
    # Criar um DataFrame com os arrays
    data = {'X_input': X.tolist(), 'Y_output': Y.tolist()}
    df = pd.DataFrame(data)
    df.to_csv(file_name, index=False, mode="a")

In [ ]:
# Para salvar arquivo grande em partes
for i in range(0,8):
    print(i)
    salvar_csv(X_train[(i*5000):((i+1)*5000)], Y_train[(i*5000):((i+1)*5000)], 'train_data50.csv') # arquivo de treino

In [ ]:
for j in range(0,2):
    print(j)
    salvar_csv(X_test[(j*5000):((j+1)*5000)], Y_test[(j*5000):((j+1)*5000)], 'test_data50.csv') # arquivo de teste

In [ ]:
# Link para download dos arquivos
from IPython.display import FileLink, FileLinks
display(FileLink('train_data.csv'))
display(FileLink('test_data.csv'))


<div id="chap4"><h1 style="color:white;background:#62909d;border-radius:5px;padding:30px;font-family:'Sans-Serif', cursive;font-size:40px;text-align:center">Compilação e Treinamento da Rede</h1></div>

**<font color="#62909d" size="5">Modelo 1 - Original U-NET</font>**

In [ ]:
# Criando uma função para um convolution block
def conv_block(inputs, num_filters):
    x = tf.keras.layers.Conv2D(num_filters, (3, 3), activation="relu", 
                               kernel_initializer="he_normal", padding="same")(inputs)
    x = tf.keras.layers.Dropout(0.1)(x)
    x = tf.keras.layers.Conv2D(num_filters, (3, 3), activation="relu", 
                               kernel_initializer="he_normal", padding="same")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    
    return x

# Criando a função para o expanding path
def upsample_block(inputs, conv_prev, num_filters):
    up = tf.keras.layers.Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding="same")(inputs)
    concat = tf.keras.layers.concatenate([up, conv_prev])
    conv = conv_block(concat, num_filters)
    return conv


In [ ]:
import tensorflow as tf

# Inputs
inputs = tf.keras.layers.Input((SIZE_X, SIZE_Y, IMG_CHANNELS))

# Normalização
s = tf.keras.layers.Lambda(lambda x: x/255.0)(inputs) 

# Contraction path
c1 = conv_block(s, 16)
p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

c2 = conv_block(p1, 32)
p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

c3 = conv_block(p2, 64)
p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)

c4 = conv_block(p3, 128)
p4 = tf.keras.layers.MaxPooling2D((2, 2))(c4)

c5 = conv_block(p4, 256)

# Expansive path
c6 = upsample_block(c5, c4, 128)
c7 = upsample_block(c6, c3, 64)
c8 = upsample_block(c7, c2, 32)
c9 = upsample_block(c8, c1, 16)

# Output layer

outputs = tf.keras.layers.Conv2D(NUM_CLASSES, (1, 1), activation='softmax')(c9)

In [ ]:
model = tf.keras.Model(inputs=[inputs], outputs=[outputs], name="U-Net")
# Compilação
opt = tf.keras.optimizers.Adam(learning_rate=0.005)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.save('multiclass_road_mapper')
model.summary()


**<font color="#62909d" size="5">Modelo 2 - Custom U-NET</font>**
<br>**Modelo sendo utilizado no momento**

In [ ]:
!pip install keras_unet

In [ ]:
from keras_unet.models import custom_unet

model = custom_unet(
    input_shape=(128, 128, 1),
    use_batch_norm=True,
    num_classes=17,
    filters=16,
    dropout=0.2,
    output_activation='softmax')


In [ ]:
model.summary()


**<font color="#62909d" size="5">Compilando o Modelo</font>**

In [ ]:
# Compilar e Salvar modelo
nome_salvar_modelo = 'multiclass_road_mapper_customunet'

opt = tf.keras.optimizers.Adam(learning_rate=0.005)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.save(nome_salvar_modelo)

In [ ]:
# Carregando o modelo
from keras.models import load_model

model = load_model('/kaggle/working/multiclass_road_mapper_customunet')
model.load_weights('/kaggle/working/multiclass_road_mapper_customunet')

**<font color="#62909d" size="5">Batch Generator</font>**

In [ ]:
from tensorflow.keras.utils import to_categorical
import json

def batch_generator(Train_df,batch_size,
                    steps):
    idx=1
    while True: 
        yield load_data(Train_df,idx-1,batch_size)## Yields data
        if idx<=steps:
            idx+=1
        else:
            idx=1
            
def load_data(Train_df,idx,
              batch_size):
    n_classes = 17

    df = pd.read_csv(
                  Train_df, skiprows=idx*batch_size,
                  nrows=batch_size)
    
    x = [] 
    y = []
    
    try:
        for i in range(0, batch_size):
            x.append(json.loads(df.iloc[i,0]))
            y.append(json.loads(df.iloc[i,1]))
    except:
        df = pd.read_csv(
                  Train_df, skiprows=0,
                  nrows=batch_size)
        for i in range(0, batch_size):
            x.append(json.loads(df.iloc[i,0]))
            y.append(json.loads(df.iloc[i,1]))
        
    y = np.asarray(y)
    train_masks_cat = to_categorical(y, num_classes=n_classes)
    y_train_cat = train_masks_cat.reshape((y.shape[0], y.shape[1], y.shape[2], n_classes))
    
    return (np.asarray(x), y_train_cat)

In [ ]:
import numpy as np

from keras.models import Sequential 
from keras.layers import Dense, Activation
batch_size = 64
nb_epoch = 2

# Objetos gerados para treino e validação
steps_per_epoch=np.ceil(40000/batch_size)
validation_steps=np.ceil(9984/batch_size)

my_training_batch_generator = batch_generator('/kaggle/input/road-mapper-dataset-csv/train_data50.csv', batch_size,steps_per_epoch)
my_validation_batch_generator = batch_generator('/kaggle/input/road-mapper-dataset-csv/test_data(1).csv', batch_size,validation_steps)


**<font color="#62909d" size="5">Treinamento</font>**

In [ ]:
# Mixed precision (Nova funcionalidade senod testada e implementada)
# https://www.tensorflow.org/guide/mixed_precision?hl=pt-br

import tensorflow as tf

try:
  gpu = tf.distribute.cluster_resolver.GPUClusterResolver() # TPU detection
except ValueError:
  gpu = None

if gpu:
  policyConfig = 'mixed_bfloat16'
else: 
  policyConfig = 'mixed_float16'

policy = tf.keras.mixed_precision.Policy(policyConfig)
tf.keras.mixed_precision.set_global_policy(policy)

In [ ]:
# Model Checkpoint e Early Stopping
# https://github.com/luddias/DeepLearning-About/blob/main/checkpointing.md
# https://github.com/luddias/DeepLearning-About/blob/main/early_stoping.md

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

filepath="/kaggle/working/multiclass_road_mapper_customunet"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
es = EarlyStopping(monitor='val_accuracy', patience=5)
callbacks_list = [checkpoint, es]

In [ ]:
# Carregando o modelo e pesos
from keras.models import load_model

model.load_weights(checkpoint_path)
model = load_model('/kaggle/working/multiclass_road_mapper')

In [ ]:
# Treinamento
import tensorflow as tf
import pandas as pd

with tf.device('/device:GPU:0'):
    history = model.fit(my_training_batch_generator,
                    epochs=nb_epoch,
                    steps_per_epoch=steps_per_epoch,
                    verbose=1, 
                    validation_data=my_validation_batch_generator,
                    validation_steps=validation_steps,
                    shuffle=True,
                    use_multiprocessing=True,
                    callbacks=[callbacks_list])


In [ ]:
# Salvando o modelo
model.save('multiclass_road_mapper_custom.keras')


In [ ]:
# Salvando os resultados
import pickle
try:  
    arquivo = open("results.bin", "wb")
    pickle.dump(history, arquivo)
    arquivo.close()
except:
    print("Problemas com o arquivo results.")
   

In [ ]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(path, download_file_name):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip {zip_name} {path} -r"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))

In [ ]:
download_file('/kaggle/working', 'out')

<div id="chap5"><h1 style="color:white;background:#62909d;border-radius:5px;padding:30px;font-family:'Sans-Serif', cursive;font-size:40px;text-align:center">Analisando os Resultados</h1></div>

In [ ]:
# Importando arquivo de resultados
import pickle

try:
    arquivo = open("/kaggle/working/results.bin", "rb")
    history = pickle.load(arquivo)
    arquivo.close()
except:
    print("Problemas com o arquivo treinox.")

In [ ]:
from matplotlib import pyplot as plt

loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

acc = history.history['categorical_accuracy']
val_acc = history.history['val_categorical_accuracy']

plt.plot(epochs, acc, 'y', label='Training Accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
plt.title('Training and validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Load do modelo salvo
from keras.models import load_model
model = load_model("/kaggle/working/multiclass_road_mapper_customunet", compile=False)

<div id="chap6"><h1 style="color:white;background:#62909d;border-radius:5px;padding:30px;font-family:'Sans-Serif', cursive;font-size:40px;text-align:center">Realizando testes (Predict)</h1></div>

In [ ]:
# Importando dados de teste
import pickle

try:
    arquivo = open("/kaggle/input/road-mapper-dnn-test-dataset/testeX50.bin", "rb")
    X_test = pickle.load(arquivo)
    arquivo.close()
except:
    print("Problemas com o arquivo treinox.")
    
try:
    arquivo = open("/kaggle/input/road-mapper-dnn-test-dataset/testeY50.bin", "rb")
    Y_test = pickle.load(arquivo)
    arquivo.close()
except:
    print("Problemas com o arquivo treinox.")

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
y_pred.shape

In [ ]:
y_pred_argmax=np.argmax(y_pred, axis=3)
y_pred_argmax.shape

In [ ]:
#Using built in keras function
from keras.metrics import MeanIoU
n_classes = 17
IOU_keras = MeanIoU(num_classes=n_classes)  
IOU_keras.update_state(Y_test[:,:,:,0], y_pred_argmax)
print("Mean IoU =", IOU_keras.result().numpy())

In [ ]:
#Calculando o I0U para cada classe...
values = np.array(IOU_keras.get_weights()).reshape(NUM_CLASSES, NUM_CLASSES)
print(values)

In [ ]:
for i in range(NUM_CLASSES):
    class_iou = values[i,i]
    soma_iou = 0.0
    for j in range(NUM_CLASSES):
        if i==j:
            soma_iou += values[i,i]
        else:
            soma_iou += values[i,j]
            soma_iou += values[j,i]
            
    class_iou = class_iou/soma_iou       
    print(f"IoU for class {i+1} is: {class_iou}")

In [ ]:
 import random
test_img_number = random.randint(0, len(X_test)-1)
test_img = X_test[test_img_number]
ground_truth=Y_test[test_img_number]
test_img_norm=test_img[:,:,0][:,:,None]
test_img_input=np.expand_dims(test_img_norm, 0)
prediction = (model.predict(test_img_input))
predicted_img=np.argmax(prediction, axis=3)[0,:,:]


plt.figure(figsize=(12, 8))
plt.subplot(231)
plt.title('Testing Image')
plt.imshow(test_img[:,:,0], cmap='gray')
plt.subplot(232)
plt.title('Testing Label')
plt.imshow(ground_truth[:,:,0], cmap='jet')
plt.subplot(233)
plt.title('Prediction on test image')
plt.imshow(predicted_img, cmap='jet')
plt.show()

In [ ]:
import json
from tensorflow.keras.utils import to_categorical
import pandas as pd
test_df = "/kaggle/input/road-mapper-dataset-csv/predict_data50(1).csv"

df = pd.read_csv(
                  test_df, skiprows=0,
                  nrows=10)

x = []
y = []

for i in range(0, 10):
    x.append(json.loads(df.iloc[i,0]))
    y.append(json.loads(df.iloc[i,1]))

x = np.asarray(x)
y = np.asarray(y)